### Working with CSV files

In [1]:
%%sh
head data/nsw_temp.csv

Product code,Bureau of Meteorology station number,Year,Month,Day,Maximum temperature (Degree C),Days of accumulation of maximum temperature,Quality
IDCJAC0010,061087,1965,01,01,25.6,,Y
IDCJAC0010,061087,1965,01,02,32.2,1,Y
IDCJAC0010,061087,1965,01,03,23.1,1,Y
IDCJAC0010,061087,1965,01,04,25.6,1,Y
IDCJAC0010,061087,1965,01,05,26.7,1,Y
IDCJAC0010,061087,1965,01,06,25.3,1,Y
IDCJAC0010,061087,1965,01,07,20.3,1,Y
IDCJAC0010,061087,1965,01,08,22.5,1,Y
IDCJAC0010,061087,1965,01,09,24.2,1,Y


In [2]:
# import pyspark class Row from module sql
from pyspark.sql import *

df_temperatures = spark.read.csv('data/nsw_temp_clean.csv')
df_temperatures.printSchema()
df_temperatures.show(5)

df_temperatures = spark.read.csv('data/nsw_temp_clean.csv', inferSchema=True, header=True)
df_temperatures.printSchema()
display(df_temperatures)

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)

+------------+--------------------+----+-----+---+--------------------+--------------------+-------+
|         _c0|                 _c1| _c2|  _c3|_c4|                 _c5|                 _c6|    _c7|
+------------+--------------------+----+-----+---+--------------------+--------------------+-------+
|Product code|Bureau of Meteoro...|Year|Month|Day|Maximum temperatu...|Days of accumulat...|Quality|
|  IDCJAC0010|              061087|1965|   01| 01|                25.6|                null|      Y|
|  IDCJAC0010|              061087|1965|   01| 02|                32.2|                   1|      Y|
|  IDCJAC0010|              061087|1965|   01| 03|                23.1|                   1|      Y|
|  IDCJ

,Product code,Bureau of Meteorology station number,Year,Month,Day,Maximum temperature (Degree C),Days of accumulation of maximum temperature,Quality
0,IDCJAC0010,61087,1965,1,1,25.6,NaN,Y
1,IDCJAC0010,61087,1965,1,2,32.2,1.0,Y
2,IDCJAC0010,61087,1965,1,3,23.1,1.0,Y
3,IDCJAC0010,61087,1965,1,4,25.6,1.0,Y
4,IDCJAC0010,61087,1965,1,5,26.7,1.0,Y
5,IDCJAC0010,61087,1965,1,6,25.3,1.0,Y
6,IDCJAC0010,61087,1965,1,7,20.3,1.0,Y
7,IDCJAC0010,61087,1965,1,8,22.5,1.0,Y
8,IDCJAC0010,61087,1965,1,9,24.2,1.0,Y
9,IDCJAC0010,61087,1965,1,10,28.6,1.0,Y


### Working with JSON files

In [3]:
%%sh
head -n 2 data/tweets.json

{"CAPTURE_ID":2,"CAPTURED_AT":"2014-05-01 00:00:35","ID":461505248142438400,"CREATED_AT":"2014-05-01 00:00:00","TEXT":"RT @allisimpson: thank you for surprising me @radiodisney &amp; @codysimpson 😊❤️ #RDMAs #RDMAafterparty #sweet16 http:\/\/t.co\/r5apnHxHAK","SCREEN_NAME":"ia_yeah","USER_ID":1100797128,"LANGUAGE":"en","PROFILE_IMAGE_URL":"http:\/\/pbs.twimg.com\/profile_images\/456806985443192832\/hFOsb_G__normal.jpeg","USER_CREATED_AT":"2013-01-18 23:12:49","COUNT":60262,"TIME_ZONE":"Hawaii","UTC_OFFSET":-36000,"FOLLOWERS":298,"FRIENDS":177,"FROM_USER_NAME":"MIDDLE FINGER☠","RETWEET_FLAG":"Y","PROFILE_IMAGE_URL_HTTPS":"https:\/\/pbs.twimg.com\/profile_images\/456806985443192832\/hFOsb_G__normal.jpeg","ORIGINAL_TWEET_ID":461505066588192768,"ORIGINAL_TWEET_LOCATION":"Gold Coast, Australia"}
{"CAPTURE_ID":2,"CAPTURED_AT":"2014-05-01 00:00:35","ID":461505250135142400,"CREATED_AT":"2014-05-01 00:00:00","TEXT":"RT @allisimpson: thank you for surprising me @radiodisney &amp; @codysimpson 😊❤️

In [4]:
df_tweets = spark.read.json('data/tweets.json')

df_tweets.printSchema()
display(df_tweets)

root
 |-- CAPTURED_AT: string (nullable = true)
 |-- CAPTURE_ID: long (nullable = true)
 |-- COUNT: long (nullable = true)
 |-- CREATED_AT: string (nullable = true)
 |-- FOLLOWERS: long (nullable = true)
 |-- FRIENDS: long (nullable = true)
 |-- FROM_USER_NAME: string (nullable = true)
 |-- ID: long (nullable = true)
 |-- IN_REPLY_TO_STATUS_ID: long (nullable = true)
 |-- LANGUAGE: string (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LOCATION: string (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- ORIGINAL_TWEET_ID: long (nullable = true)
 |-- ORIGINAL_TWEET_LOCATION: string (nullable = true)
 |-- PROFILE_IMAGE_URL: string (nullable = true)
 |-- PROFILE_IMAGE_URL_HTTPS: string (nullable = true)
 |-- RETWEET_FLAG: string (nullable = true)
 |-- SCREEN_NAME: string (nullable = true)
 |-- TEXT: string (nullable = true)
 |-- TIME_ZONE: string (nullable = true)
 |-- TO_USER: string (nullable = true)
 |-- TO_USER_ID: long (nullable = true)
 |-- USER_CREATED_AT

,CAPTURED_AT,CAPTURE_ID,COUNT,CREATED_AT,FOLLOWERS,FRIENDS,FROM_USER_NAME,ID,IN_REPLY_TO_STATUS_ID,LANGUAGE,...,PROFILE_IMAGE_URL_HTTPS,RETWEET_FLAG,SCREEN_NAME,TEXT,TIME_ZONE,TO_USER,TO_USER_ID,USER_CREATED_AT,USER_ID,UTC_OFFSET
0,2014-05-01 00:00:35,2,60262,2014-05-01 00:00:00,298,177,MIDDLE FINGER☠,461505248142438400,NaN,en,...,https://pbs.twimg.com/profile_images/456806985...,Y,ia_yeah,RT @allisimpson: thank you for surprising me @...,Hawaii,None,NaN,2013-01-18 23:12:49,1100797128,-36000.0
1,2014-05-01 00:00:35,2,15707,2014-05-01 00:00:00,974,415,Nina Mustifasari,461505250135142400,NaN,en,...,https://pbs.twimg.com/profile_images/460202598...,Y,NinaMustifasari,RT @allisimpson: thank you for surprising me @...,Jakarta,None,NaN,2012-04-15 20:34:55,554329827,25200.0
2,2014-05-01 00:00:35,2,22131,2014-05-01 00:00:01,3115,370,Amber (◕‿◕✿),461505252017975296,NaN,en,...,https://pbs.twimg.com/profile_images/460048163...,N,ambershutup,I also really was sour cream,Brisbane,None,NaN,2011-07-06 15:56:18,330160256,36000.0
3,2014-05-01 00:00:35,2,12028,2014-05-01 00:00:01,2183,1905,it's me,461505254681772032,NaN,it,...,https://pbs.twimg.com/profile_images/456885461...,N,ciaonash,Divergevnt è carino,Athens,None,NaN,2012-09-01 05:07:07,794477280,10800.0
4,2014-05-01 00:00:35,2,3015,2014-05-01 00:00:01,894,1138,Renee,461505254685540353,NaN,en,...,https://pbs.twimg.com/profile_images/461095110...,N,waetforder_1,@Usher Can you please follow me? x,None,Usher,4.090893e+07,2014-01-04 23:00:31,2276018148,NaN
5,2014-05-01 00:00:35,2,27291,2014-05-01 00:00:01,895,598,Nico Suavè,461505254828544000,NaN,en,...,https://pbs.twimg.com/profile_images/460481514...,Y,NICKa_please23,"RT @CoachN_J_: Since when did saying ""female"" ...",Quito,None,NaN,2010-04-28 00:54:37,137717608,-18000.0
6,2014-05-01 00:00:35,2,824,2014-05-01 00:00:01,154,195,Masafumi Asian,461505255444721666,4.615045e+17,en,...,https://pbs.twimg.com/profile_images/461487799...,N,MasafumiWatana1,@U_ki310 @D8128 \nこれガチやで笑\n笑いとまらん,None,U_ki310,5.715836e+08,2013-12-28 05:49:01,2264774749,NaN
7,2014-05-01 00:00:35,2,22505,2014-05-01 00:00:02,560,221,Dellɑ Agesty L,461505256606560257,NaN,en,...,https://pbs.twimg.com/profile_images/456841248...,Y,DellaAgesty,RT @allisimpson: thank you for surprising me @...,Pacific Time (US & Canada),None,NaN,2011-06-11 13:45:19,314984878,-25200.0
8,2014-05-01 00:00:35,2,5185,2014-05-01 00:00:02,537,268,wwat,461505256623325186,NaN,en,...,https://pbs.twimg.com/profile_images/460550021...,Y,starbxs,RT @itskirrabitch: Liam's Instagram pics are k...,Bangkok,None,NaN,2013-12-30 15:45:24,2268335251,25200.0
9,2014-05-01 00:00:35,2,8079,2014-05-01 00:00:02,1050,2001,13 Days || 8 Days,461505257537687553,4.615046e+17,en,...,https://pbs.twimg.com/profile_images/446601795...,N,SunshineCalx,@goddamnitluke @tomlinclique_ to leave just si...,Brisbane,goddamnitluke,1.964238e+09,2012-04-26 20:03:41,563600126,36000.0


In [5]:
from pyspark.sql.functions import col, desc
df_tweets_by_location = df_tweets.groupBy('ORIGINAL_TWEET_LOCATION').count().sort(desc('count')).limit(10)
display(df_tweets_by_location)

,ORIGINAL_TWEET_LOCATION,count
0,None,3489
1,"Gold Coast, Australia",699
2,NC,284
3,"Brisbane, Australia",131
4,Brisbane,64
5,Nc,34
6,Surfers Paradise,21
7,Emerald City,14
8,Townsville,13
9,The City of Townsville,12


In [6]:
df_tweets_by_location.write.csv('/tmp/tweets_by_location.csv', mode='overwrite', header=True)

In [7]:
%%sh
ls -l /tmp/tweets_by_location.csv
head -n 5 /tmp/tweets_by_location.csv/part*

total 8
-rw-r--r--  1 szu004  wheel    0  5 Jul 14:00 _SUCCESS
-rw-r--r--  1 szu004  wheel  191  5 Jul 14:00 part-00000-d953308f-9993-4a23-ab8a-17c31fc76369.csv
ORIGINAL_TWEET_LOCATION,count
,3489
"Gold Coast, Australia",699
NC,284
"Brisbane, Australia",131


In [8]:
import re
df_renamed = df_temperatures.select(*[col(c).alias(re.sub(r'[ ()]','_', c)) for c in df_temperatures.columns])
df_renamed.printSchema()
df_renamed.write.parquet('/tmp/nsw_temp.parquet')

root
 |-- Product_code: string (nullable = true)
 |-- Bureau_of_Meteorology_station_number: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Day: integer (nullable = true)
 |-- Maximum_temperature__Degree_C_: double (nullable = true)
 |-- Days_of_accumulation_of_maximum_temperature: integer (nullable = true)
 |-- Quality: string (nullable = true)



In [9]:
%%sh
ls -l /tmp/nsw_temp.parquet
echo
du -hs /tmp/nsw_temp.parquet
echo 
ls -lh data/nsw_temp_clean.csv

total 840
-rw-r--r--  1 szu004  wheel       0  5 Jul 14:00 _SUCCESS
-rw-r--r--  1 szu004  wheel  136038  5 Jul 14:00 part-00000-fff7a509-ff59-4ea4-b362-641e7e5ac97b.snappy.parquet
-rw-r--r--  1 szu004  wheel  135871  5 Jul 14:00 part-00001-fff7a509-ff59-4ea4-b362-641e7e5ac97b.snappy.parquet
-rw-r--r--  1 szu004  wheel  138299  5 Jul 14:00 part-00002-fff7a509-ff59-4ea4-b362-641e7e5ac97b.snappy.parquet
-rw-r--r--  1 szu004  wheel   10894  5 Jul 14:00 part-00003-fff7a509-ff59-4ea4-b362-641e7e5ac97b.snappy.parquet

440K	/tmp/nsw_temp.parquet

-rw-r--r--  1 szu004  staff    13M  4 Jul 13:37 data/nsw_temp_clean.csv


http://spark.apache.org/docs/2.1.0/api/python/pyspark.sql.html#pyspark.sql.DataFrame